In [18]:
!pip install markupsafe==2.0.1 Jinja2

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#install geoSpatial libraries needed
!pip install geopandas
!pip install folium

In [8]:
# import geospatial libraries
import geopandas as gpd
import fiona

# import other libraries
import pandas as pd
import math

# import plotting libraries
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from shapely.geometry import Point, Polygon

In [4]:
# Create a map
lincolnshire = folium.Map(location=[53.234444, -0.538611], tiles='openstreetmap', zoom_start=8)

# Display the map
lincolnshire

In [9]:
# enable KML support which is disabled by default
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [10]:
# plot lincolnshire force KML boundary file
# read the KML file
lincolnshire_force_boundary = gpd.read_file('data/lincolnshire_force_boundary.kml', driver='KML')

# plot the KML file over the map
folium.GeoJson(lincolnshire_force_boundary).add_to(lincolnshire)

# Display the map
lincolnshire

In [12]:
# read the 2023-07-lincolnshire-street.csv file
lincolnshire_street = pd.read_csv('data/2023-07-lincolnshire-street.csv', low_memory=False)

# display the first 5 rows of the dataframe
lincolnshire_street.head(3)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,e3b4f1befe578d1b475926b286a195034a63f928a7a65d...,2023-07,Lincolnshire Police,Lincolnshire Police,0.061405,52.984299,On or near Tyler Crescent,E01026007,Boston 001A,Violence and sexual offences,Under investigation,NaN
1,fb0c36dd9fb5c4ef6728b84f6d0e4851e85cf47feef82f...,2023-07,Lincolnshire Police,Lincolnshire Police,0.062130,52.985131,On or near Brand End Road,E01026007,Boston 001A,Violence and sexual offences,Under investigation,NaN
2,NaN,2023-07,Lincolnshire Police,Lincolnshire Police,0.061304,52.981234,On or near St Andrew'S Road,E01026008,Boston 001B,Anti-social behaviour,NaN,NaN


In [29]:
#check for missing values
lincolnshire_street.isnull().sum()

Crime ID                 1086
Month                       0
Reported by                 0
Falls within                0
Longitude                   0
Latitude                    0
Location                    0
LSOA code                   0
LSOA name                   0
Crime type                  0
Last outcome category    1086
Context                  5917
dtype: int64

In [30]:
# get value count of crime type
lincolnshire_street['Crime type'].value_counts()

Violence and sexual offences    2142
Anti-social behaviour           1086
Shoplifting                      532
Criminal damage and arson        475
Other theft                      390
Public order                     376
Burglary                         268
Vehicle crime                    187
Other crime                      149
Drugs                            120
Bicycle theft                     84
Possession of weapons             59
Robbery                           32
Theft from the person             17
Name: Crime type, dtype: int64

In [31]:
# check for Violence and sexual offences crime type without crime ID
lincolnshire_street[(lincolnshire_street['Crime type'] == 'Violence and sexual offences') & (lincolnshire_street['Crime ID'].isnull())]

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context


In [ ]:
# All Violence and sexual offences crime have crime ID
# Lets analyze Violence and sexual offences crime type being the most common crime type as VSO

In [13]:
lincolnshire_street[lincolnshire_street['Crime type'] == 'Violence and sexual offences']

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,e3b4f1befe578d1b475926b286a195034a63f928a7a65d...,2023-07,Lincolnshire Police,Lincolnshire Police,0.061405,52.984299,On or near Tyler Crescent,E01026007,Boston 001A,Violence and sexual offences,Under investigation,NaN
1,fb0c36dd9fb5c4ef6728b84f6d0e4851e85cf47feef82f...,2023-07,Lincolnshire Police,Lincolnshire Police,0.062130,52.985131,On or near Brand End Road,E01026007,Boston 001A,Violence and sexual offences,Under investigation,NaN
3,b0ac74bdce1fa722c4fcc39b00124e1f9cad41215e6837...,2023-07,Lincolnshire Police,Lincolnshire Police,0.062546,52.983218,On or near Centenary Close,E01026008,Boston 001B,Violence and sexual offences,Under investigation,NaN
9,01d2e779a983c5f6435c800e6e3e180328b67b9c96bc00...,2023-07,Lincolnshire Police,Lincolnshire Police,0.052375,53.021955,On or near Ings Road,E01026009,Boston 001C,Violence and sexual offences,Under investigation,NaN
12,6631a98979bb4186062eb749f6314b5f0af758c4a27c7d...,2023-07,Lincolnshire Police,Lincolnshire Police,0.125710,53.037201,On or near Oak Court,E01026023,Boston 001D,Violence and sexual offences,Unable to prosecute suspect,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5904,e0d38cd034b8f83f96d9937bbf76c1024a014af8fa23c0...,2023-07,Lincolnshire Police,Lincolnshire Police,-0.444201,53.251637,On or near Church Lane,E01026371,West Lindsey 011D,Violence and sexual offences,Under investigation,NaN
5913,5c72ebd6dec0f72765ffde21857c4ee99ff338ce83603d...,2023-07,Lincolnshire Police,Lincolnshire Police,-0.425744,53.234629,On or near Church View Crescent,E01026373,West Lindsey 011E,Violence and sexual offences,Under investigation,NaN
5914,e0e088093b975d837f8b3424c7fcd5b37c5409e56b455c...,2023-07,Lincolnshire Police,Lincolnshire Police,-0.404136,53.275757,On or near Main Road,E01026373,West Lindsey 011E,Violence and sexual offences,Unable to prosecute suspect,NaN
5915,cd2e2ff691d8da90358f7336bc8adb6748868a93f4f753...,2023-07,Lincolnshire Police,Lincolnshire Police,-0.425744,53.234629,On or near Church View Crescent,E01026373,West Lindsey 011E,Violence and sexual offences,Under investigation,NaN


In [14]:
# create a subset of the dataframe with only the columns we need for Violence and sexual offences crime type
lincolnshire_street_vso = lincolnshire_street[lincolnshire_street['Crime type'] == 'Violence and sexual offences']

In [15]:
# add points markers to the map for Violence and sexual offences crime type
for index, row in lincolnshire_street_vso.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(lincolnshire)

# Display the map
lincolnshire

In [16]:
# create the map
lincolnshire = folium.Map(location=[53.234444, -0.538611], tiles='openstreetmap', zoom_start=8)

# add marker clusters
# initialize the cluster
mc = MarkerCluster()
# add points markers to the map for Violence and sexual offences crime type
for index, row in lincolnshire_street_vso.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
            mc.add_child(Marker([row['Latitude'], row['Longitude']]))
# Display the map
lincolnshire.add_child(mc)

In [ ]:
#The map is now clustered and we can see the number of crimes in each cluster
# The higher numbers indicate more crimes in that area with the highest (355) occuring in Lincoln central area.
# The more focal a town is the more crimes it has.
# This can also be seen using a heatmap

In [17]:
# create the map
lincolnshire = folium.Map(location=[53.234444, -0.538611], tiles='openstreetmap', zoom_start=8)

# add heatmap to base map
HeatMap(data=lincolnshire_street_vso[['Latitude', 'Longitude']], radius=15).add_to(lincolnshire)

# Display the map
lincolnshire